In [1]:
import apache_beam as beam
import typing

In [9]:
class Employee(object):
    def __init__(self, id, name):
        self.id = id
        self.name = name

In [10]:
class EmployeeCoder(beam.coders.Coder):

    def encode(self, employee):
        return ('%s:%s' % (employee.id, employee.name)).encode('utf-8')

    def decode(self, s):
        return Employee(*s.decode('utf-8').split(':'))

    def is_deterministic(self):
        return True

In [11]:
beam.coders.registry.register_coder(Employee, EmployeeCoder)

In [18]:
@beam.typehints.with_output_types(typing.Tuple[Employee, int])
def split_file(input):
    imput = input.split(',')
    id, name, salary = input[0], input[1], input[2]
    return Employee(id, name), int(salary)

In [19]:
p = beam.Pipeline()

result = (
    p
    | beam.io.ReadFromText("dept_data.txt")
    | beam.Map(split_file)
    | beam.CombinePerKey(sum).with_input_types(typing.Tuple[Employee, int])
)

p.run()	